In [1]:
from quantumML.rest import MWRester

In [2]:
b=MWRester()


# Queries
- **band_gap_range**  
Range of bandgaps in *eV* as a two valued list eg. [1.2, 3.1]  
- **Formation_energy_range**  
Range of formation energies in *meV* as two values list eg. [50, 150]
- **elements**  
List of elements contained in entry eg. ['Na', 'Cl']  
note: this will return all entries that contain Na and Cl as well as any other element 
- **space_group_number**  
integer of space group number eg. 5
- **dimention**  
integer of dimension of material eg. 2
- **crystal_system**  
crystal system as a string eg. 'Tetragonal'  


In [3]:
help(b.get_calculation)

Help on method get_calculation in module quantumML.rest:

get_calculation(band_gap_range=None, formation_energy_range=None, elements=[], space_group_number=None, dimension=None, crystal_system=None, name=None) method of quantumML.rest.MWRester instance
    Method to that queries materialsweb database and returns a list of dictionaries of calculations that
    fit the querries parameters. Additionally
    Parameters:
        band_gap_range (list): List of band gap range e.g. [min, max]
        formation_energy_range (list): List of formation energy range e.g. [min, max]
        elements (list): List of str of elements
        space_group_number (int): space group number
        dimension (int): dimension as int e.g. 1 2 3
        crystal_system (str): name of crystal system
        name (str): name of material e.g. MoS2
    Returns:
        results: List of results matching quire parameters



In [4]:
d = b.get_calculation(band_gap_range=[1,1.5], formation_energy_range=[124,150])
for d in b.results:
    #print(d.keys())
    if len(d['composition'].split('/')[-2].replace('%',''))< 8:
        print(d['composition'].split('/')[-2].replace('%','')+'\t\t\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')
    elif len(d['composition'].split('/')[-2].replace('%',''))< 16:
        print(d['composition'].split('/')[-2].replace('%','')+'\t\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')
    else:
        print(d['composition'].split('/')[-2].replace('%','')+'\t BandGap = ' + \
              str(round(d['band_gap'],4))+' eV\t Formation Eneergy = ' + str(d['formation_energy'])+ ' meV')


Pd120S2			 BandGap = 1.1746 eV	 Formation Eneergy = 148.0 meV
Ge120Se1		 BandGap = 1.2107 eV	 Formation Eneergy = 133.0 meV
Pd120Se2		 BandGap = 1.3964 eV	 Formation Eneergy = 166.0 meV
I220Pt1			 BandGap = 1.4215 eV	 Formation Eneergy = 135.0 meV
I220Pd1			 BandGap = 1.1512 eV	 Formation Eneergy = 133.0 meV
Br320Cr1		 BandGap = 1.4714 eV	 Formation Eneergy = 742.0 meV
Cr120H120O2		 BandGap = 1.0072 eV	 Formation Eneergy = 179.0 meV
Mo120O3			 BandGap = 1.1381 eV	 Formation Eneergy = 136.0 meV


# Write files

In [5]:
help(b.write)
#help(b.write_all)

Help on method write in module quantumML.rest:

write(index=0) method of quantumML.rest.MWRester instance
    Writes INCAR, KPOINTS, POSCAR of entry to current directory
    
    Parameters:
        index (int): index of entry to write files for



In [6]:
import os 
os.mkdir('example')
%cd example
print(type(b))
b.write_all()
%ls
%cd ..

FileExistsError: [Errno 17] File exists: 'example'

In [ ]:
%cd ~/dev/mwinterface
%pwd

# Machine Learning

In [7]:
#Convert to list of pymatgen structures

c = b.as_pymatgen_struc()

In [8]:
MWRester().get_symmetry_functions_g2(c[0])
b.get_soap(b.results[0])

array([[ 3.23715732e-02,  4.29601129e-03,  3.62247764e-03,
        -1.82748225e-03,  6.71045855e-03,  5.72189747e-04,
         4.98028065e-04, -2.51956750e-04,  8.52994621e-04,
         5.49888762e-04, -2.83340196e-04,  4.37105395e-04,
         1.46175546e-04, -2.07636374e-04,  2.07245979e-03,
         4.84641851e-06, -1.27033127e-05,  7.06361743e-06,
         1.84710243e-05,  4.33171999e-06,  3.33478856e-05,
        -1.84206419e-05, -4.91428691e-05, -1.06311736e-05,
         1.28466208e-05,  2.68671120e-05, -5.54062217e-06,
         8.16366737e-05, -1.23714744e-06,  8.77736602e-05,
         3.66675499e-06, -1.02916611e-05,  1.22379970e-06,
         2.65821218e-05,  1.30838416e-05,  2.95985719e-05,
        -2.10575607e-07, -8.84286492e-05, -2.97435090e-05,
         3.44540713e-05, -6.00897511e-05, -5.35784675e-05,
         4.63482313e-04, -1.40571256e-05,  5.42705297e-04,
         9.64503761e-06, -2.70058135e-05,  2.48923261e-05,
         3.16124606e-05, -9.66213793e-06,  7.56741429e-0

Query materialsproject for all Cd-Te 

In [9]:
from pymatgen import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
import os
from pymatgen.io.vasp import inputs
os.mkdir('mldir')
os.chdir('mldir')
a = MPRester('7JGUQgNZyOTTp8Tc')
entries = a.query('Cd-Te',properties = ['structure','formation_energy_per_atom'])
#entries = a.query('Cd-Te',properties = ['formation_energy_per_atom'])
formation_e_actual=[]
i= 0 
for b in entries:
    b['structure'].to(fmt='poscar',filename=str(i)+'.poscar')
    formation_e_actual.append(b['formation_energy_per_atom'])
    i = i +1 
entries = a.query('Cd',properties = ['structure','formation_energy_per_atom'])
for b in entries:
    b['structure'].to(fmt='poscar',filename=str(i)+'.poscar')
    formation_e_actual.append(b['formation_energy_per_atom'])
    i = i +1 
entries = a.query('Te',properties = ['structure','formation_energy_per_atom'])
for b in entries:
    b['structure'].to(fmt='poscar',filename=str(i)+'.poscar')
    formation_e_actual.append(b['formation_energy_per_atom'])
    i = i +1 


Retreive pre-trained ML model from materialsweb

In [13]:
formation_e_predict=[]
model=MWRester().get_KRR('Cd-Te')
for i in range(0,23):
    #print(i)
    formation_e_predict.append(model.predict(MWRester().get_soap('./'+str(i)+'.poscar',fmt='poscar'))[0])

http://2dmaterialsweb.org/static/models/Cd-Te.sav


/home/jason/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVR from version 0.23.1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


AttributeError: 'SVR' object has no attribute 'probA_'

In [ ]:
!ls
!mkdir temp

In [ ]:
!cd temp
b.prep_ml_formation_energy(b.results[-1])

In [ ]:
!ls

In [ ]:
def get_soap(calculation, rcut=7, nmax=5, lmax=8, fmt='MW'):
    if fmt == 'MW':
        urlp = 'http://2dmaterialsweb.org/' + calculation['path'][22:] + '/POSCAR'
        file = urllib.request.urlopen(urlp)
        file = file.read().decode("utf-8")
        file = io.StringIO(file)
    elif fmt == 'poscar':
        file = calculation
    ml=vasp.read_vasp(file)
    species=['Cd','Te']
    periodic_soap = SOAP(
    periodic=True,
    species=np.unique(ml.get_atomic_numbers()),
    rcut=rcut,
    nmax=nmax,
    lmax=lmax,
    rbf='gto',
    sigma=0.125,
    average=True
    )
    soap = periodic_soap.create(ml)
    #soap = 1
    return soap

In [ ]:
get_soap( b.results[0])